In [22]:
import collections
import pandas as pd
from functools import lru_cache
import itertools

In [1]:
with open('../inputs/input16.txt', 'r') as f:
    lines = f.readlines()
n = len(lines)
valves = []
for i in range(n):
    lines[i] = lines[i].strip()
    lines[i] = lines[i].split(";")
    lines[i][1] = lines[i][1].replace(",", " ")
    lines[i][1] = lines[i][1].split()
    valves.append([lines[i][0][6:8], lines[i][0][23:], lines[i][1][4:]])
lines

[['Valve JZ has flow rate=0', ['tunnels', 'lead', 'to', 'valves', 'IR', 'LY']],
 ['Valve KD has flow rate=0', ['tunnels', 'lead', 'to', 'valves', 'NJ', 'ZS']],
 ['Valve VW has flow rate=0', ['tunnels', 'lead', 'to', 'valves', 'IT', 'VH']],
 ['Valve HS has flow rate=0', ['tunnels', 'lead', 'to', 'valves', 'OC', 'PN']],
 ['Valve EU has flow rate=19', ['tunnel', 'leads', 'to', 'valve', 'GQ']],
 ['Valve XF has flow rate=0', ['tunnels', 'lead', 'to', 'valves', 'WL', 'QD']],
 ['Valve DD has flow rate=8',
  ['tunnels', 'lead', 'to', 'valves', 'GQ', 'YY', 'JV', 'SK']],
 ['Valve TA has flow rate=0', ['tunnels', 'lead', 'to', 'valves', 'NJ', 'VJ']],
 ['Valve IR has flow rate=9',
  ['tunnels', 'lead', 'to', 'valves', 'JZ', 'WI', 'VJ', 'GC', 'WG']],
 ['Valve SS has flow rate=17',
  ['tunnels', 'lead', 'to', 'valves', 'SI', 'IZ', 'RK', 'WI']],
 ['Valve SG has flow rate=0', ['tunnels', 'lead', 'to', 'valves', 'NV', 'NJ']],
 ['Valve IT has flow rate=0', ['tunnels', 'lead', 'to', 'valves', 'LL', 'VW']

In [2]:
for valve in valves:
    valve[1] = int(valve[1])
valves

[['JZ', 0, ['IR', 'LY']],
 ['KD', 0, ['NJ', 'ZS']],
 ['VW', 0, ['IT', 'VH']],
 ['HS', 0, ['OC', 'PN']],
 ['EU', 19, ['GQ']],
 ['XF', 0, ['WL', 'QD']],
 ['DD', 8, ['GQ', 'YY', 'JV', 'SK']],
 ['TA', 0, ['NJ', 'VJ']],
 ['IR', 9, ['JZ', 'WI', 'VJ', 'GC', 'WG']],
 ['SS', 17, ['SI', 'IZ', 'RK', 'WI']],
 ['SG', 0, ['NV', 'NJ']],
 ['IT', 0, ['LL', 'VW']],
 ['CP', 24, ['HN', 'ZK', 'EJ']],
 ['SK', 0, ['LL', 'DD']],
 ['IS', 0, ['AA', 'LL']],
 ['HN', 0, ['FF', 'CP']],
 ['VH', 10, ['QO', 'VW', 'RV', 'PN']],
 ['JV', 0, ['DD', 'RK']],
 ['ZS', 0, ['KD', 'LL']],
 ['UC', 25, ['JD', 'IV']],
 ['WI', 0, ['SS', 'IR']],
 ['UR', 0, ['QD', 'LY']],
 ['GC', 0, ['AA', 'IR']],
 ['YY', 0, ['DD', 'AA']],
 ['IV', 0, ['ZK', 'UC']],
 ['BM', 0, ['SA', 'WL']],
 ['JD', 0, ['IZ', 'UC']],
 ['WL', 12, ['EF', 'BM', 'EJ', 'XF']],
 ['AA', 0, ['NV', 'YY', 'GC', 'IS', 'QO']],
 ['WG', 0, ['LL', 'IR']],
 ['GQ', 0, ['EU', 'DD']],
 ['SI', 0, ['SS', 'NJ']],
 ['KH', 13, ['SA', 'ON']],
 ['PC', 22, ['ON']],
 ['QD', 14, ['XF', 'UR']],
 ['

In [3]:
rates = {}
for valve in valves:
    rates[valve[0]] = valve[1]
rates

{'JZ': 0,
 'KD': 0,
 'VW': 0,
 'HS': 0,
 'EU': 19,
 'XF': 0,
 'DD': 8,
 'TA': 0,
 'IR': 9,
 'SS': 17,
 'SG': 0,
 'IT': 0,
 'CP': 24,
 'SK': 0,
 'IS': 0,
 'HN': 0,
 'VH': 10,
 'JV': 0,
 'ZS': 0,
 'UC': 25,
 'WI': 0,
 'UR': 0,
 'GC': 0,
 'YY': 0,
 'IV': 0,
 'BM': 0,
 'JD': 0,
 'WL': 12,
 'AA': 0,
 'WG': 0,
 'GQ': 0,
 'SI': 0,
 'KH': 13,
 'PC': 22,
 'QD': 14,
 'IZ': 0,
 'QO': 0,
 'SA': 0,
 'NV': 0,
 'ZK': 0,
 'ON': 0,
 'PN': 0,
 'RV': 0,
 'RK': 0,
 'OC': 18,
 'EF': 0,
 'VJ': 0,
 'LL': 5,
 'FF': 0,
 'LY': 21,
 'EJ': 0,
 'NJ': 6}

In [4]:
adjacency_lists = {}
for valve in valves:
    adjacency_lists[valve[0]] = valve[2]
adjacency_lists

{'JZ': ['IR', 'LY'],
 'KD': ['NJ', 'ZS'],
 'VW': ['IT', 'VH'],
 'HS': ['OC', 'PN'],
 'EU': ['GQ'],
 'XF': ['WL', 'QD'],
 'DD': ['GQ', 'YY', 'JV', 'SK'],
 'TA': ['NJ', 'VJ'],
 'IR': ['JZ', 'WI', 'VJ', 'GC', 'WG'],
 'SS': ['SI', 'IZ', 'RK', 'WI'],
 'SG': ['NV', 'NJ'],
 'IT': ['LL', 'VW'],
 'CP': ['HN', 'ZK', 'EJ'],
 'SK': ['LL', 'DD'],
 'IS': ['AA', 'LL'],
 'HN': ['FF', 'CP'],
 'VH': ['QO', 'VW', 'RV', 'PN'],
 'JV': ['DD', 'RK'],
 'ZS': ['KD', 'LL'],
 'UC': ['JD', 'IV'],
 'WI': ['SS', 'IR'],
 'UR': ['QD', 'LY'],
 'GC': ['AA', 'IR'],
 'YY': ['DD', 'AA'],
 'IV': ['ZK', 'UC'],
 'BM': ['SA', 'WL'],
 'JD': ['IZ', 'UC'],
 'WL': ['EF', 'BM', 'EJ', 'XF'],
 'AA': ['NV', 'YY', 'GC', 'IS', 'QO'],
 'WG': ['LL', 'IR'],
 'GQ': ['EU', 'DD'],
 'SI': ['SS', 'NJ'],
 'KH': ['SA', 'ON'],
 'PC': ['ON'],
 'QD': ['XF', 'UR'],
 'IZ': ['SS', 'JD'],
 'QO': ['AA', 'VH'],
 'SA': ['BM', 'KH'],
 'NV': ['AA', 'SG'],
 'ZK': ['CP', 'IV'],
 'ON': ['PC', 'KH'],
 'PN': ['HS', 'VH'],
 'RV': ['NJ', 'VH'],
 'RK': ['SS', 'JV']

In [8]:
paths = {}
def bfs(vertex):
    Q = collections.deque()
    visited = set()
    visited.add(vertex)
    Q.append((vertex,0))
    while (Q):
        v,dist = Q.popleft()
        path_name = vertex + '-' + v
        paths[path_name] = dist
        for edge in adjacency_lists[v]:
            if edge not in visited:
                Q.append((edge, dist+1))
                visited.add(edge)
        

In [9]:
for valve in valves:
    bfs(valve[0])
paths

{'JZ-JZ': 0,
 'JZ-IR': 1,
 'JZ-LY': 1,
 'JZ-WI': 2,
 'JZ-VJ': 2,
 'JZ-GC': 2,
 'JZ-WG': 2,
 'JZ-EF': 2,
 'JZ-FF': 2,
 'JZ-UR': 2,
 'JZ-SS': 3,
 'JZ-TA': 3,
 'JZ-AA': 3,
 'JZ-LL': 3,
 'JZ-WL': 3,
 'JZ-HN': 3,
 'JZ-QD': 3,
 'JZ-SI': 4,
 'JZ-IZ': 4,
 'JZ-RK': 4,
 'JZ-NJ': 4,
 'JZ-NV': 4,
 'JZ-YY': 4,
 'JZ-IS': 4,
 'JZ-QO': 4,
 'JZ-ZS': 4,
 'JZ-IT': 4,
 'JZ-SK': 4,
 'JZ-BM': 4,
 'JZ-EJ': 4,
 'JZ-XF': 4,
 'JZ-CP': 4,
 'JZ-JD': 5,
 'JZ-JV': 5,
 'JZ-RV': 5,
 'JZ-KD': 5,
 'JZ-SG': 5,
 'JZ-DD': 5,
 'JZ-VH': 5,
 'JZ-VW': 5,
 'JZ-SA': 5,
 'JZ-ZK': 5,
 'JZ-UC': 6,
 'JZ-GQ': 6,
 'JZ-PN': 6,
 'JZ-KH': 6,
 'JZ-IV': 6,
 'JZ-EU': 7,
 'JZ-HS': 7,
 'JZ-ON': 7,
 'JZ-OC': 8,
 'JZ-PC': 8,
 'KD-KD': 0,
 'KD-NJ': 1,
 'KD-ZS': 1,
 'KD-RV': 2,
 'KD-SG': 2,
 'KD-SI': 2,
 'KD-TA': 2,
 'KD-LL': 2,
 'KD-VH': 3,
 'KD-NV': 3,
 'KD-SS': 3,
 'KD-VJ': 3,
 'KD-IT': 3,
 'KD-SK': 3,
 'KD-IS': 3,
 'KD-WG': 3,
 'KD-QO': 4,
 'KD-VW': 4,
 'KD-PN': 4,
 'KD-AA': 4,
 'KD-IZ': 4,
 'KD-RK': 4,
 'KD-WI': 4,
 'KD-IR': 4,
 'KD-DD': 4,

In [10]:
#Function for weights
def w(u, v):
    path_name = u+'-'+v
    return paths[path_name]

In [11]:
open_valves = []
for valve in valves:
    if valve[1] != 0:
        open_valves.append(valve[0])
open_valves.sort()
open_valves

['CP',
 'DD',
 'EU',
 'IR',
 'KH',
 'LL',
 'LY',
 'NJ',
 'OC',
 'PC',
 'QD',
 'SS',
 'UC',
 'VH',
 'WL']

In [17]:
def pressure_released(start, end, time_remaining):
    distance = w(start,end)
    time_remaining -= distance+1
    return time_remaining*rates[end]

In [18]:
def t_remaining(start, end, time_remaining):
    distance = w(start,end)
    time_remaining -= distance+1
    return time_remaining

In [21]:
#Answer to Part 1
still_open = (1,1,1,1,1,1,1,1,1,1,1,1,1,1,1)
@lru_cache(maxsize=None)
def dfs(still_open, current_location, time_remaining):
    if time_remaining <= 0:
        return 0
    max_pressure = 0
    for i in range(15):
        if still_open[i] == 1:
            temp = list(still_open)
            temp[i] = 0
            temp = tuple(temp)
            max_pressure = max(max_pressure, pressure_released(current_location, open_valves[i], time_remaining) + dfs(temp, open_valves[i], t_remaining(current_location, open_valves[i], time_remaining)))
    return max_pressure
dfs(still_open, 'AA', 30)
    

2056

In [32]:
#Answer to Part 2
human_combinations = itertools.combinations([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14], 8)
global_max = 0
for combo in human_combinations:
    human_tuple = [0]*15
    elephant_tuple = [1]*15
    for index in combo:
        human_tuple[index] = 1
        elephant_tuple[index] = 0
    human_tuple = tuple(human_tuple)
    elephant_tuple = tuple(elephant_tuple)
    global_max = max(global_max, dfs(human_tuple, 'AA', 26)+dfs(elephant_tuple, 'AA', 26))
global_max

2513